# Examples to make plots of countries

In order make plots of the World (and subsets of it), we use data from natural earth (https://www.naturalearthdata.com/) which are in the public domain. They are available at different scales and with different levels of details.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

### Plotting using GeoPandas

In [ ]:
import geopandas as gpd

import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

# Read shape file  - get country borders
resolution = '110m' #'10m'
category = 'cultural'
name = 'admin_0_countries'

# read the natural earth shapefile using geopandas
shpfilename = shpreader.natural_earth(resolution, category, name)
world = gpd.read_file(shpfilename)

world.plot()

In [ ]:
# plot only one continent
africa = world[world['CONTINENT'] == "Africa"]
africa.plot()

# save as shapefile
# africa.to_file("africa.shp")

In [ ]:
# plot only one country
namibia = africa[africa['ADMIN'] == "Namibia"]
namibia.plot()

# save as shapefile
# namibia.to_file("namibia.shp")
# read from file
# df = gpd.read_file("namibia.shp")

In [ ]:
# plot continent and highlight one country
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=cartopy.crs.Mollweide())
plt.axis('off')
africa.plot(ax=ax, edgecolor="black", facecolor='lightgray', lw=0.25)
namibia.plot(ax=ax, edgecolor="None", facecolor='red', lw=0.25)

### Plotting with Cartopy

In [ ]:
import cartopy.feature as cfeature


def basemap_germany():
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111)

    extent = [5.8, 15, 47, 55.2]
    central_lon = np.mean(extent[:2])
    central_lat = np.mean(extent[2:])

    ax = plt.axes(projection=ccrs.Orthographic(central_lon, central_lat))
    ax.set_extent(extent)

    highlight=['Germany']
    shpfilename = shpreader.natural_earth(resolution='10m', category='cultural', name='admin_0_map_subunits')
    reader = shpreader.Reader(shpfilename)
    countries = reader.records()

    for country in countries:
         if country.attributes['NAME'] in highlight:
                ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                                  facecolor=cfeature.COLORS['land'], edgecolor='grey')

    ax.add_feature(cfeature.LAKES)
    ax.add_feature(cfeature.RIVERS)
    ax.add_feature(cfeature.OCEAN)

    lakes_EU = cfeature.NaturalEarthFeature(category='physical',name='lakes_europe',
                                            scale='10m',facecolor=cfeature.COLORS['water'])
    ax.add_feature(lakes_EU)    
    
    
    return fig, ax

fig, ax = basemap_germany()

In [ ]:
shape_df = gpd.read_file('plz-5stellig.shp/vg2500_12-31.tm32.shape/vg2500/VG2500_KRS.shp', dtype={'plz': str})
shape_df.head()

In [ ]:
plz_shape_df = gpd.read_file('plz-5stellig.shp/plz-5stellig.shp', dtype={'plz': str})
plz_shape_df.head()

plt.rcParams['figure.figsize'] = [16, 11]

# Get lat and lng of Germany's main cities. 
top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Köln': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'München': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nürnberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052)
}

fig, ax = plt.subplots()

plz_shape_df.plot(ax=ax, color=cfeature.COLORS['land'])

# Plot cities. 
for c in top_cities.keys():
    # Plot city name.
    ax.text(
        x=top_cities[c][0], 
        # Add small shift to avoid overlap with point.
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )
    # Plot city location centroid.
    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    aspect=1.3, 
    facecolor=cfeature.COLORS['water']
);

In [ ]:
# Create feature.
plz_shape_df = plz_shape_df \
    .assign(first_dig_plz = lambda x: x['plz'].str.slice(start=0, stop=1))

In [ ]:
fig, ax = plt.subplots()

plz_shape_df.plot(
    ax=ax, 
    column='first_dig_plz', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'First Digit', 'loc':'lower right'},
    cmap='tab20',
    alpha=0.9
)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )

ax.set(
    title='Germany First-Digit-Postal Codes Areas', 
    aspect=1.3,
    facecolor='white'
);

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np

plz_region_df = pd.read_csv(
    'plz-5stellig.shp/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)

plz_region_df.drop('osm_id', axis=1, inplace=True)

plz_region_df.head()

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)

germany_df.drop(['note'], axis=1, inplace=True)

germany_df.head()

In [ ]:
fig, ax = plt.subplots()

germany_df.plot(
    ax=ax, 
    column='bundesland', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'Bundesland', 'bbox_to_anchor': (1.35, 0.8)},
#     cmap='tab20'
)

# for c in top_cities.keys():

#     ax.text(
#         x=top_cities[c][0], 
#         y=top_cities[c][1] + 0.08, 
#         s=c, 
#         fontsize=12,
#         ha='center', 
#     )

#     ax.plot(
#         top_cities[c][0], 
#         top_cities[c][1], 
#         marker='o',
#         c='black', 
#         alpha=0.5
#     )

ax.set(
    aspect=1.3, 
    facecolor='white'
);